In [21]:
import pandas as pd
import numpy as np
import math
import datetime
import subprocess
import bandit.main as bd

In [22]:
def get_frame(filename):
    df = pd.read_excel(io=filename)
    # df_combined = pd.merge(df,df_lookup,on='Difficulty')
    # df_combined['Duration'] = df_combined['Time'].apply(lambda x: pd.Timedelta(str(x))).astype('timedelta64[s]')
    # df_combined['MatRate'] = df_combined['Mats'] / df_combined['Duration']
    # df_combined['DiffClass'] = df_combined['Class'] + '-' + df_combined['Difficulty']
    return df

In [23]:
def save_excel(df,col, madfunc):
    dfgb = df.groupby([col])
    dfgb = dfgb['Quality'].agg(['count','median', madfunc]).rename(columns={'mad_function': 'MAD'}).reset_index()
    print(dfgb)
    return dfgb

In [24]:
subprocess.Popen('EQTime.xlsx', shell=True)

In [25]:
df = get_frame('EQTime.xlsx')
df.tail(10)

,ProjeDate,Project,Duration
25,2018-08-24,PCAutoMechanic,00:52:19
26,2018-08-26,PCAutoMechanic,00:32:11
27,2018-08-31,ShadowOfWar,00:52:53
28,2018-09-03,ShadowOfWar,00:19:29
29,2018-09-08,DualBoxing,00:03:00
30,2018-09-16,ShadowOfWar,00:37:20
31,2018-09-22,PCAutoMechanic,00:50:05
32,2018-09-23,PCAutoMechanic,00:15:54
33,2018-10-05,Beastlord,00:39:14
34,2018-10-08,PCAutoMechanic,00:01:27


In [26]:
# reduce it
df = bd.reduce(df,'Project')
df['Duration'] = df['Duration'].apply(lambda x: pd.Timedelta(str(x))).astype('timedelta64[s]')
df_quality = bd.get_quality(df['Duration'])
df = df.join(df_quality)

25 0.9375 23


In [27]:
df.tail(10)

,ProjeDate,Project,Duration,Quality
25,2018-08-24,PCAutoMechanic,3139.0,0.963080
26,2018-08-26,PCAutoMechanic,1931.0,0.581887
27,2018-08-31,ShadowOfWar,3173.0,0.973809
28,2018-09-03,ShadowOfWar,1169.0,0.341433
29,2018-09-08,DualBoxing,180.0,0.029347
30,2018-09-16,ShadowOfWar,2240.0,0.679394
31,2018-09-22,PCAutoMechanic,3005.0,0.920795
32,2018-09-23,PCAutoMechanic,954.0,0.273588
33,2018-10-05,Beastlord,2354.0,0.715368
34,2018-10-08,PCAutoMechanic,87.0,0.000000


In [28]:
df.sort_values('Quality', ascending=False).head(10)

,ProjeDate,Project,Duration,Quality
18,2018-07-29,ShadowOfWar,3256.0,1.000000
27,2018-08-31,ShadowOfWar,3173.0,0.973809
25,2018-08-24,PCAutoMechanic,3139.0,0.963080
31,2018-09-22,PCAutoMechanic,3005.0,0.920795
22,2018-08-11,ShadowOfWar,2986.0,0.914800
20,2018-08-05,PCAutoMechanic,2559.0,0.780057
33,2018-10-05,Beastlord,2354.0,0.715368
3,2018-04-29,Mage,2313.0,0.702430
5,2018-05-06,Witcher,2290.0,0.695172
30,2018-09-16,ShadowOfWar,2240.0,0.679394


In [29]:
dfgb = save_excel(df,'Project', bd.mad_function)

          Project  count    median       MAD
0            Bard      2  0.317293  0.135532
1       Beastlord      3  0.476491  0.015778
2      DualBoxing      2  0.227517  0.198170
3      ForTheKing      2  0.404386  0.018145
4        KerbalSP      2  0.345377  0.235248
5            Mage      3  0.430420  0.272010
6           Necro      2  0.275008  0.098927
7  PCAutoMechanic      7  0.581887  0.308299
8     ShadowOfWar      7  0.679394  0.235405
9         Witcher      3  0.396971  0.204481


In [30]:
df = df.merge(dfgb)
dfgb['total'] = dfgb['count'].sum()
dfgb['donext'] = dfgb.apply(lambda x: bd.get_tuned_ucb(x, 'median', 'MAD'), axis=1)
dfgb.sort_values('donext', ascending=False)

,Project,count,median,MAD,total,donext
3,ForTheKing,2,0.404386,0.018145,33,1.065494
8,ShadowOfWar,7,0.679394,0.235405,33,1.032771
1,Beastlord,3,0.476491,0.015778,33,1.016283
4,KerbalSP,2,0.345377,0.235248,33,1.006485
0,Bard,2,0.317293,0.135532,33,0.978400
5,Mage,3,0.430420,0.272010,33,0.970212
9,Witcher,3,0.396971,0.204481,33,0.936763
6,Necro,2,0.275008,0.098927,33,0.936116
7,PCAutoMechanic,7,0.581887,0.308299,33,0.935264
2,DualBoxing,2,0.227517,0.198170,33,0.888624
